In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy
from wavy.model import compute_default_scores
# from wavy.plot import plot_dataframes
from wavy.utils import reverse_pct_change

In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

# get stock info
# msft.info

In [5]:
# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [6]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,19.040895,19.147468,18.934323,18.998266,65002900,0.0,0
2005-01-04,19.090632,19.254042,18.941430,19.069317,109442100,0.0,0
2005-01-05,19.069318,19.254043,19.012480,19.026690,72463500,0.0,0
2005-01-06,19.076416,19.225616,18.927214,19.005367,76890500,0.0,0
2005-01-07,19.055112,19.104845,18.913016,18.948540,68723300,0.0,0
...,...,...,...,...,...,...,...
2022-04-13,282.730011,288.579987,281.299988,287.619995,21907200,0.0,0
2022-04-14,288.089996,288.309998,279.320007,279.829987,28107900,0.0,0
2022-04-18,278.910004,282.459991,278.339996,280.519989,20778000,0.0,0


In [7]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [8]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [9]:
df = hist.pct_change()

In [10]:
df.plot()

In [11]:
print(len(df) - len(df.dropna()))
df.dropna(inplace=True)

1


In [12]:
df.High.hist(bins=200)

In [13]:
x, y = wavy.create_panels(df, lookback=10, horizon=1)

In [24]:
frame1 = x[1:3, 'High']
frame2 = x[3:5, 'High']

TypeError: list indices must be integers or slices, not tuple

In [21]:
res = frame1 + frame2

In [23]:
res[0]

,Open,High,Low,Close
Date,,,,
2005-01-05,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN
2005-01-07,-0.002234,-0.012564,-0.001500,-0.005980
2005-01-10,-0.016406,-0.002231,-0.006011,0.009749
2005-01-11,0.006767,-0.002979,0.005275,-0.005224
2005-01-12,0.005994,0.002237,0.000751,0.003741
2005-01-13,-0.006724,-0.003724,-0.034560,-0.038088
2005-01-14,-0.020989,-0.026119,-0.009174,-0.011420
2005-01-18,-0.028031,-0.007562,-0.015362,0.015313


In [ ]:
frame[0]

In [ ]:
frame[['High']]

In [ ]:
type(frame)

In [ ]:
from wavy.plot import add_bar_trace, add_scatter_trace, add_lineball_trace, split_background, panel_plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [ ]:
data = y[['High']].as_dataframe()

In [ ]:
fig = go.Figure()

panel_plot(y, 'High')

In [ ]:
from wavy.plot import panel_plot

fig = panel_plot(y, 'High')

In [ ]:
# predict only the highest price
y = y[['High']]

In [ ]:
# try single col inputs first
x_high = x[['High']]

In [ ]:
from wavy.plot import panel_plot

panel_plot(x_high, 'High')

In [ ]:
lr = wavy.LinearRegression(x_high, y)

In [ ]:
lr.fit(epochs=10)
y_pred = lr.predict()

In [ ]:
baseline = wavy.BaselineConstant(x_high, y, model_type="regression", constant=0)
baseline.fit()

In [ ]:
y_pred = baseline.predict()

In [ ]:
rev_y = reverse_pct_change(y, hist)
rev_y_pred = reverse_pct_change(y_pred, hist)

In [ ]:
baseline = wavy.BaselineShift(x_high, y, model_type="regression")
baseline.fit()

In [ ]:
baseline.score()

In [ ]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

In [ ]:
y_pred = baseline.predict()

In [ ]:
panel_plot(y_pred)

In [ ]:
y_pred.plot()

In [ ]:
rev_y = reverse_pct_change(y, hist)
rev_y_pred = reverse_pct_change(y_pred, hist)

In [ ]:
baseline.score().test

In [ ]:
residuals = baseline.residuals

In [ ]:
panel_plot(residuals) # Highest error is in train set

In [ ]:
# # TODO: 

# fig = go.Figure()

# def line_component(panel, fig):
#     line_plot(fig, data, col)
#     split_background(fig, data)
#     return fig

# def bar_component(panel, fig):
#     bar_plot(fig, data, col)
#     split_background(fig, data)
#     return fig

# ...



# compile_plot(fig)

In [ ]:
lr = wavy.LinearRegression(x_high, y)
lr.fit(epochs=20)

In [ ]:
panel_plot(lr.residuals())

In [ ]:
dense = wavy.DenseModel(x_high, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
dense.plot_residuals()

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression", conv_layers=1, conv_filters=32,kernel_size=3, dense_layers=1, dense_units=32)
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
params=[{'conv_layers': [1, 5, 10], 'conv_filters': [16, 32], 'kernel_size': [2, 3, 7], 'dense_layers': [1], 'dense_units': [32]}, {'dense_layers': 1, 'dense_units': 32}]

models = [ConvModel, DenseModel]

# wavy.GridSearchModel(x, y, model_type, models=, )

In [ ]:
lr.score()

In [ ]:
grid_search --> sklearn

In [ ]:
conv.plot_residuals()

In [ ]:
# TODO: Compare residuals between models

In [ ]:
default_scores = compute_default_scores(x, y, model_type="regression", metrics=["MAE", "MSE"])

In [ ]:
# rev_y = reverse_pct_change(y, hist).as_dataframe()
# rev_y_conv = reverse_pct_change(conv.predict(), hist).as_dataframe()
# rev_y_base = reverse_pct_change(baseline.predict(), hist).as_dataframe()

# rev_y_conv.columns = ['conv_' + col for col in rev_y_conv.columns]
# rev_y_base.columns = ['base_' + col for col in rev_y_base.columns]

In [ ]:
# plot_dataframes([rev_y, rev_y_conv, rev_y_base])

In [ ]:
y_pred[0]

In [ ]:
hist.iloc[10:20]

In [ ]:
y_orig.as_dataframe()

In [ ]:
y.as_dataframe()

In [ ]:
y_orig = y.update(hist.shift())

In [ ]:
y_orig[0]

In [ ]:
y_pred_orig = y_pred.shift() + y_orig

In [ ]:
y_pred_orig.plot()

In [ ]:
y_true = y.test.as_dataframe()
y_baseline = baseline.predict().as_dataframe()
y_lr = lr.predict().as_dataframe()
y_dense = dense.predict().as_dataframe()
y_conv = conv.predict().as_dataframe()

y_true.columns = ['true_' + col for col in y_true.columns]
y_baseline.columns = ['baseline_' + col for col in y_baseline.columns]
y_lr.columns = ['lr_' + col for col in y_lr.columns]
y_dense.columns = ['dense_' + col for col in y_dense.columns]
y_conv.columns = ['conv_' + col for col in y_conv.columns]

plot_dataframes([y_true, y_baseline, y_lr, y_dense, y_conv])

In [ ]:
baseline

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)

In [ ]:

# data1 = {'train': x.train.as_dataframe()["High"],
#         'val': x.val.as_dataframe()["High"],
#         'test': x.test.as_dataframe()["High"]
#         }

# data2 = {'train': x.train.as_dataframe()["Low"],
#         'val': x.val.as_dataframe()["Low"],
#         'test': x.test.as_dataframe()["Low"]
#         }

In [ ]:
aaa = y[-1000:]

s1 = y.train.as_dataframe()['High']
s2 = y.val.as_dataframe()['High']
s3 = y.test.as_dataframe()['High']

In [ ]:
fig = Figure()

In [ ]:
# fig.linebar(s)
# fig.scatter(s)

fig.dotbar(s1)
fig.dotbar(s2, color="blue")
fig.dotbar(s3)

In [ ]:
fig.fig

In [ ]:
fig = PanelFigure()
fig.split_sets(data1)
fig.add_area(data1)
fig.add_dotbar(data2)

In [ ]:
fig.compile()